In [37]:
import os
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import psycopg2
import streamlit as st
from dotenv import load_dotenv
from pandas import json_normalize

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

### Historico de estoque fulfillment ###
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    # sql_query = f"SELECT * FROM mcenter_fulfillment_stock WHERE created_at BETWEEN '{date_from}' AND '{date_to};'"
    sql_query = f"SELECT * FROM mcenter_fulfillment_stock WHERE created_at BETWEEN '2023-11-4' AND '2023-12-14';"
    # print(sql_query)
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar mcenter_fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar mcenter_fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

# Ordenando stock por data
df_stock = df_stock.sort_values(by="created_at", ascending=False)
df_stock["data"] = df_stock["created_at"].dt.date
df_stock = df_stock.drop(["created_at"], axis=1)
df_stock.shape

C:\Users\couti\AppData\Local\Temp\ipykernel_10740\61461247.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


(11284, 7)

In [40]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_has_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_has_stock = df_has_stock.sort_values(by="data", ascending=False).reset_index(
    drop=True
)
df_has_stock.shape

(11284, 8)

In [42]:
## Contando dias em que produto esteve disponível
days_available = (
    df_has_stock.groupby(["ml_inventory_id"])["has_stock"].sum().reset_index()
)
days_available = days_available.rename(columns={"has_stock": "days_available"})
days_available.shape
# days_available

,ml_inventory_id,days_available
0,ABCB20467,31
1,AGDL67534,31
2,AGDL67547,31
3,AIRQ38676,31
4,AJZG23550,31
...,...,...
359,ZXJO91694,31
360,ZZGN21450,31
361,ZZQL21216,31
362,ZZRV30730,31


In [44]:
# Unindo DFs
df_stock_days = df_has_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
df_stock_days.shape

(11284, 9)

In [45]:
df_stock_days.sample(2)

,ml_inventory_id,available_quantity,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available
9840,BZEY80870,8,None,NaN,MLB2004757564,174305087698,2023-11-30,True,31
2233,IYUQ03362,0,None,NaN,MLB2000350479,0,2023-12-12,False,0


In [46]:
# data de hoje
data_de_hoje = datetime.now().date()
data_de_hoje = data_de_hoje - timedelta(days=1)
# print(data_de_hoje)

df_stock_days["data"] = pd.to_datetime(df_stock_days["data"])

# Filtra apenas as linhas onde 'data' é igual à data de hoje
df_stock_today = df_stock_days[df_stock_days["data"].dt.date == data_de_hoje]
df_stock_today = df_stock_today.rename(
    columns={"available_quantity": "available_quantity_today"}
)

# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity
df_stock_today["total_available_quantity"] = df_stock_today.apply(
    lambda row: row["detail_quantity"] + row["available_quantity_today"]
    if row["detail_status"] == "transfer"
    else row["available_quantity_today"],
    axis=1,
)

df_stock_today["total_available_quantity"] = df_stock_today[
    "total_available_quantity"
].astype("int64")

In [47]:
df_stock_today.sample(2)

,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available,total_available_quantity
5735,YHXD60928,49,None,NaN,MLB2986538160,175888357629,2023-12-13,True,31,49
2046,QRSL03944,0,None,NaN,MLB2000967077,nan,2023-12-13,False,0,0


In [ ]:
### Buscando hitorico de orders no BD ###

# Buscando histórico de vendas na tabela mcenter_ml_orders para o período definido
try:
    conn = psycopg2.connect(**db_config)

    # sql_query = f"SELECT * FROM mcenter_ml_orders WHERE date_closed BETWEEN '{date_from}' AND '{date_to}'"
    sql_query = f"SELECT * FROM mcenter_ml_orders WHERE date_closed BETWEEN '2023-11-4' AND '2023-12-14';"
    # print(sql_query)
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar mcenter_ml_orders: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar mcenter_ml_orders: {e}")

except Exception as e:
    print(f"Erro ao consultar mcenter_ml_orders: {e}")
    # logger.error(f"Erro ao consultar mcenter_ml_orders: {e}")

finally:
    if conn is not None:
        conn.close()

# filtros
df_orders = df_orders[df_orders["logistic_type"] == "fulfillment"]
# df_orders = df_orders.drop(columns=['category_id','pack_id','variation_attributes_id','variation_name','variation_value_id', 'data'])
df_orders = df_orders.drop(
    columns=[
        "category_id",
        "pack_id",
        "variation_attributes_id",
        "variation_name",
        "variation_value_id",
    ]
)

# change column
df_orders["variation_id"] = df_orders["variation_id"].replace("nan", "0", regex=True)

df_orders.rename(columns={"quantity": "sold_quantity"}, inplace=True)

# print(df_orders.shape)
df_orders = df_orders.drop_duplicates()
# print(df_orders.shape)

# Ordenando orders por data
df_orders = df_orders.sort_values(by="date_approved", ascending=False)
df_orders["data"] = df_orders["date_approved"].dt.date
df_orders = df_orders.drop(["date_closed", "date_approved"], axis=1)

# Total de vendas por ml_code e id de variação
df_orders_quantity = (
    df_orders.groupby(["ml_code", "variation_id"])["sold_quantity"].sum().reset_index()
)

print(f"Número de ml_code únicos: {len(df_orders_quantity['ml_code'].unique())}")
print(
    f"Número de variation_id únicos: {len(df_orders_quantity['variation_id'].unique())}"
)

# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(
    # df_orders, resultado, on=["ml_code", "variation_id"], how="inner"
    df_orders,
    df_orders_quantity,
    on=["ml_code", "variation_id"],
    how="inner",
)
df_total_sales = df_total_sales.rename(
    columns={"sold_quantity_y": "total_sold_quantity"}
)
df_total_sales = df_total_sales.drop(
    columns=["sold_quantity_x", "order_status", "payment_status"]
)


# Buscando dados de produtos na tabela mcenter_items
try:
    conn = psycopg2.connect(**db_config)
    sql_query = "SELECT * FROM mcenter_items"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar mcenter_fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar mcenter_items: {e}")

except Exception as e:
    # logger.error(f"Erro ao consultar tabela mcenter_items: {e}")
    print(f"Erro ao consultar tabela mcenter_items: {e}")

finally:
    if conn is not None:
        conn.close()

# df_codes["ml_code"] = df_codes["ml_code"].apply(lambda x: "MLB" + str(x))
df_codes.rename(columns={"inventory_id": "ml_inventory_id"}, inplace=True)
df_codes = df_codes.drop(["created_at", "updated_at"], axis=1)

df_not_catalogo = df_codes[df_codes["catalog_listing"] == False]
df_catalogo = df_codes[df_codes["catalog_listing"] == True]

df_total_sales_cat = pd.merge(
    df_catalogo,
    df_total_sales,
    left_on=["ml_code"],
    right_on=["ml_code"],
    how="left",
)
df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_not_cat = pd.merge(
    df_not_catalogo,
    df_total_sales,
    left_on=["ml_code", "variation_id"],
    right_on=["ml_code", "variation_id"],
    how="left",
)
df_total_sales_not_cat = df_total_sales_not_cat.drop_duplicates()

df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].fillna(0)
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].fillna(0)

df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].astype("int64")
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].astype("int64")

df_total_sales_cat_x = df_total_sales_cat.drop(
    columns=["data", "shipping_id", "variation_id_x", "order_id"]
)
df_total_sales_cat_x = df_total_sales_cat_x.drop_duplicates()

df_total_sales_not_cat_x = df_total_sales_not_cat.drop(
    columns=["data", "shipping_id", "order_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat_x.drop_duplicates()

df_total_sales_cat_x = df_total_sales_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)

df_total_cat = df_total_sales_cat_x.copy()
df_total_not_cat = df_total_sales_not_cat_x.copy()

# Somando total de vendas por inventory_id
df_sum_qt_sold_cat = (
    df_total_cat.groupby("ml_inventory_id")["total_sold_quantity"].sum().reset_index()
)
df_sum_qt_sold_cat = df_sum_qt_sold_cat.rename(
    columns={"total_sold_quantity": "total_sold_catalog"}
)

df_total_cat = pd.merge(
    df_total_cat,
    df_sum_qt_sold_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)

df_total_cat.rename(columns={"variation_id_y": "variation_id_"})

df_sum_qt_sold_not_cat = (
    df_total_not_cat.groupby("ml_inventory_id")["total_sold_quantity"]
    .sum()
    .reset_index()
)
df_sum_qt_sold_not_cat = df_sum_qt_sold_not_cat.rename(
    columns={"total_sold_quantity": "total_sold_not_catalog"}
)
df_total_not_cat = pd.merge(
    df_total_not_cat,
    df_sum_qt_sold_not_cat[["ml_inventory_id", "total_sold_not_catalog"]],
    on="ml_inventory_id",
    how="left",
)


df_total_cat.rename(columns={"variation_id_y": "variation_id"}, inplace=True)

df_total_cat = df_total_cat.drop_duplicates(subset=["ml_inventory_id"])

df_combined = pd.merge(
    df_total_not_cat,
    df_total_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)

df = pd.merge(df_combined, df_stock_today, on="ml_inventory_id", how="left")
df["total_sold_catalog"] = df["total_sold_catalog"].fillna(0).astype("int64")

days = input_days
days = 30

df["total_sold"] = df["total_sold_catalog"] + df["total_sold_not_catalog"]

# qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df["period_send_fulfillment"] = np.ceil(
    (df["total_sold"] / df["days_available"]) * days - df["total_available_quantity"]
)

df["period_send_fulfillment"] = df["period_send_fulfillment"].fillna(0)


def calculate_percentual_send(row):
    if row["days_available"] != 0:
        if np.ceil(
            (row["total_sold"] / row["days_available"]) * days * 0.7
            > row["total_available_quantity"]
        ):
            # return (np.ceil(row["total_sold"] / row["days_available"]) * days - row["total_available_quantity"])
            return np.ceil(
                (row["total_sold"] / row["days_available"]) * days
                - row["total_available_quantity"]
            )

    return 0


# Aplicando a função à coluna "percentual_send"
df["stock_replenishment"] = df.apply(calculate_percentual_send, axis=1)

df_have_itens = df[df["days_available"] > 0]

# produtos sem estoque no período
df_no_itens = df[df["days_available"] <= 0]
df_no_itens = df_no_itens.drop(columns=["period_send_fulfillment"])

df_sold_zero = df_have_itens[df_have_itens["total_sold"] == 0]
df_sold = df_have_itens[df_have_itens["total_sold"] > 0]

dfx = df_have_itens.copy()

cols = [
    "ml_code",
    "seller_sku",
    "ml_inventory_id",
    "value_name",
    "status",
    "title",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "total_available_quantity",
    "days_available",
    "total_sold_not_catalog",
    "total_sold_catalog",
    "total_sold",
    "period_send_fulfillment",
    "stock_replenishment",
]

df_sold_zero = df_sold_zero[cols]
df_sold = df_sold[cols]


def rename_columns(df):
    return df.rename(
        columns={
            "detail_status": "transfer_status",
            "detail_quantity": "transfer_quantity",
        }
    )


df_sold_zero = rename_columns(df_sold_zero)
df_sold = rename_columns(df_sold)
df_no_itens = rename_columns(df_no_itens)

In [34]:
df_sold_zero
df_stock_today

,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available,total_available_quantity
0,OKDI38047,0,None,NaN,MLB2003184492,nan,2023-12-13,False,0,0
31,UITT86657,0,None,NaN,MLB1734684354,6999131408,2023-12-13,False,0,0
62,GHFV89567,13,lost,1.0,MLB1738251798,nan,2023-12-13,True,1,13
93,JOGC74297,66,None,NaN,MLB1738241921,nan,2023-12-13,True,31,66
124,JOGC18300,30,None,NaN,MLB1813392401,7772688918,2023-12-13,True,31,30
...,...,...,...,...,...,...,...,...,...,...
11129,RPIM55972,3,None,NaN,MLB2130684657,nan,2023-12-13,True,31,3
11160,ZNFW24350,0,None,NaN,MLB1662066408,64423053648,2023-12-13,False,0,0
11191,TIOX38546,2,None,NaN,MLB2006294912,nan,2023-12-13,True,31,2
11222,WOJB37093,2,None,NaN,MLB1971185030,93313756459,2023-12-13,True,31,2


In [35]:
df_codes

,ml_code,ml_inventory_id,value_name,variation_id,status,catalog_listing
0,MLB3778561802,AVOX05934,NaN,NaN,closed,True
1,MLB3436515963,WTHI04163,NaN,NaN,closed,True
2,MLB3778538730,JLAX10536,NaN,NaN,closed,True
3,MLB924922735,JFGN34621,NaN,NaN,paused,False
4,MLB949771924,TDFV00990,NaN,NaN,paused,False
...,...,...,...,...,...,...
526,MLB3387475661,MAEG54107,nan,NaN,active,True
527,MLB3427769549,UIUW89800,nan,NaN,active,True
528,MLB3497394073,XGSW66673,nan,NaN,active,True
529,MLB3516894673,PPWL59058,nan,NaN,active,True
